# Kaggle Airbnb - Preprocessing Test

In [1]:
import glob

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB, MultinomialNB

from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import mean_squared_error

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

from sklearn.cross_validation import LeaveOneOut, cross_val_score
import sklearn.cross_validation as cv

from matplotlib import pyplot as plt
%matplotlib inline

In [175]:
test = pd.read_csv('../../Kaggle/Airbnb/test_users.csv', \
                         parse_dates=['date_account_created', 'timestamp_first_active', 'date_first_booking'])

In [176]:
test.shape

(62096, 15)

In [177]:
test.columns

Index(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser'],
      dtype='object')

In [178]:
test['dac_dayofweek'] = [i.dayofweek for i in test['date_account_created']]
test['dac_day'] = [i.dayofyear for i in test['date_account_created']]
test['dac_month'] = [i.month for i in test['date_account_created']]
test['dac_year'] = [i.year for i in test['date_account_created']]
test['tfa_dayofweek'] = [i.dayofweek for i in test['timestamp_first_active']]
test['tfa_day'] = [i.dayofyear for i in test['timestamp_first_active']]
test['tfa_month'] = [i.month for i in test['timestamp_first_active']]
test['tfa_year'] = [i.year for i in test['timestamp_first_active']]
test['tfa_hour'] = [i.hour for i in test['timestamp_first_active']]

In [179]:
print(pd.unique(test['gender']))
gd = pd.get_dummies(test['gender'])
test = test.merge(gd, suffixes=('', ''),
                            left_index=True, right_index=True)

['FEMALE' '-unknown-' 'MALE' 'OTHER']


In [180]:
print(list(test.head()))

['id', 'date_account_created', 'timestamp_first_active', 'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser', 'dac_dayofweek', 'dac_day', 'dac_month', 'dac_year', 'tfa_dayofweek', 'tfa_day', 'tfa_month', 'tfa_year', 'tfa_hour', '-unknown-', 'FEMALE', 'MALE', 'OTHER']


In [181]:
test.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,...,dac_year,tfa_dayofweek,tfa_day,tfa_month,tfa_year,tfa_hour,-unknown-,FEMALE,MALE,OTHER
0,5uwns89zht,2014-07-01,2014-07-01 00:00:06,NaT,FEMALE,35,facebook,0,en,direct,...,2014,1,182,7,2014,0,0,1,0,0
1,jtl0dijy2j,2014-07-01,2014-07-01 00:00:51,NaT,-unknown-,NaN,basic,0,en,direct,...,2014,1,182,7,2014,0,1,0,0,0
2,xx0ulgorjt,2014-07-01,2014-07-01 00:01:48,NaT,-unknown-,NaN,basic,0,en,direct,...,2014,1,182,7,2014,0,1,0,0,0
3,6c6puo6ix0,2014-07-01,2014-07-01 00:02:15,NaT,-unknown-,NaN,basic,0,en,direct,...,2014,1,182,7,2014,0,1,0,0,0
4,czqhjk3yfe,2014-07-01,2014-07-01 00:03:05,NaT,-unknown-,NaN,basic,0,en,direct,...,2014,1,182,7,2014,0,1,0,0,0


In [182]:
test['date_first_booking'].unique()

array(['NaT'], dtype='datetime64[ns]')

In [183]:
test['age'] = test['age'].apply(lambda x: (2016 - x) if x > 1916 else x)

In [184]:
test[test['age'] > 97] = np.NaN
test[test['age'] < 15] = np.NaN

In [185]:
test['age'].mean()

34.20706733106466

In [186]:
test['age'] = test['age'].fillna(test['age'].mean())

In [187]:
print(pd.unique(test['signup_method']))
sm = pd.get_dummies(test['signup_method'])
test = test.merge(sm, suffixes=('', ''),
                            left_index=True, right_index=True)

['facebook' 'basic' nan 'google' 'weibo']


In [188]:
list(test.columns)

['id',
 'date_account_created',
 'timestamp_first_active',
 'date_first_booking',
 'gender',
 'age',
 'signup_method',
 'signup_flow',
 'language',
 'affiliate_channel',
 'affiliate_provider',
 'first_affiliate_tracked',
 'signup_app',
 'first_device_type',
 'first_browser',
 'dac_dayofweek',
 'dac_day',
 'dac_month',
 'dac_year',
 'tfa_dayofweek',
 'tfa_day',
 'tfa_month',
 'tfa_year',
 'tfa_hour',
 '-unknown-',
 'FEMALE',
 'MALE',
 'OTHER',
 'basic',
 'facebook',
 'google',
 'weibo']

In [189]:
print(pd.unique(test['language']))
lang = pd.get_dummies(test['language'])
test = test.merge(lang, suffixes=('test', ''),
                            left_index=True, right_index=True)

['en' 'de' 'zh' 'fr' nan 'ko' 'sv' 'no' 'it' 'es' 'nl' 'ja' 'ru' 'pt' 'tr'
 'cs' 'el' 'hu' 'pl' 'da' 'fi' 'th' 'ca' '-unknown-' 'id']


In [190]:
print(pd.unique(test['affiliate_channel']))
ac = pd.get_dummies(test['affiliate_channel'])
test = test.merge(ac, suffixes=('', ''),
                            left_index=True, right_index=True)

['direct' 'sem-brand' 'sem-non-brand' 'seo' 'remarketing' nan 'other'
 'content']


In [191]:
print(pd.unique(test['affiliate_provider']))
ap = pd.get_dummies(test['affiliate_provider'])
test = test.merge(ap, suffixes=('test', ''),
                            left_index=True, right_index=True)

['direct' 'google' 'bing' 'facebook' nan 'other' 'craigslist' 'padmapper'
 'email-marketing' 'yahoo' 'baidu' 'naver' 'gsp' 'facebook-open-graph'
 'meetup' 'vast' 'daum' 'yandex']


In [192]:
print(pd.unique(test['first_affiliate_tracked']))
fat = pd.get_dummies(test['first_affiliate_tracked'])
test = test.merge(fat, suffixes=('', ''),
                            left_index=True, right_index=True)

['untracked' 'linked' 'omg' 'product' 'marketing' nan 'tracked-other'
 'local ops']


In [193]:
print(pd.unique(test['signup_app']))
sa = pd.get_dummies(test['signup_app'])
test = test.merge(sa, suffixes=('', ''),
                            left_index=True, right_index=True)

['Moweb' 'Web' 'iOS' 'Android' nan]


In [194]:
print(pd.unique(test['first_device_type']))
fdt = pd.get_dummies(test['first_device_type'])
test = test.merge(fdt, suffixes=('', ''),
                            left_index=True, right_index=True)

['iPhone' 'Windows Desktop' 'Mac Desktop' 'iPad' 'Android Tablet'
 'Android Phone' nan 'Desktop (Other)' 'Other/Unknown' 'SmartPhone (Other)']


In [195]:
print(pd.unique(test['first_browser']))
test.first_browser = test.first_browser.str.replace('-unknown-', 'notknown')
print('------')
print(pd.unique(test['first_browser']))
fb = pd.get_dummies(test['first_browser'])
test = test.merge(fb, suffixes=('test', ''),
                            left_index=True, right_index=True)

['Mobile Safari' 'Chrome' 'IE' 'Safari' '-unknown-' 'Firefox'
 'Chrome Mobile' 'Android Browser' nan 'IE Mobile' 'BlackBerry Browser'
 'Opera' 'Silk' 'Mobile Firefox' 'AOL Explorer' 'SeaMonkey' 'Opera Mobile'
 'wOSBrowser' 'Chromium' 'Apple Mail' 'Maxthon' 'IBrowse' 'Sogou Explorer'
 'Iron' 'Yandex.Browser' 'SiteKiosk' 'Pale Moon' 'Nintendo Browser'
 'Opera Mini' 'CometBird' 'IceWeasel' 'UC Browser']
------
['Mobile Safari' 'Chrome' 'IE' 'Safari' 'notknown' 'Firefox'
 'Chrome Mobile' 'Android Browser' nan 'IE Mobile' 'BlackBerry Browser'
 'Opera' 'Silk' 'Mobile Firefox' 'AOL Explorer' 'SeaMonkey' 'Opera Mobile'
 'wOSBrowser' 'Chromium' 'Apple Mail' 'Maxthon' 'IBrowse' 'Sogou Explorer'
 'Iron' 'Yandex.Browser' 'SiteKiosk' 'Pale Moon' 'Nintendo Browser'
 'Opera Mini' 'CometBird' 'IceWeasel' 'UC Browser']


In [196]:
print(list(test.columns))

['idtest', 'date_account_created', 'timestamp_first_active', 'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser', 'dac_dayofweek', 'dac_day', 'dac_month', 'dac_year', 'tfa_dayofweek', 'tfa_day', 'tfa_month', 'tfa_year', 'tfa_hour', '-unknown-test', 'FEMALE', 'MALE', 'OTHER', 'basic', 'facebooktest', 'googletest', 'weibo', '-unknown-', 'ca', 'cs', 'da', 'de', 'el', 'en', 'es', 'fi', 'fr', 'hu', 'id', 'it', 'ja', 'ko', 'nl', 'no', 'pl', 'pt', 'ru', 'sv', 'th', 'tr', 'zh', 'content', 'directtest', 'othertest', 'remarketing', 'sem-brand', 'sem-non-brand', 'seo', 'baidu', 'bing', 'craigslist', 'daum', 'direct', 'email-marketing', 'facebook', 'facebook-open-graph', 'google', 'gsp', 'meetup', 'naver', 'other', 'padmapper', 'vast', 'yahoo', 'yandex', 'linked', 'local ops', 'marketing', 'omg', 'product', 'tracked-other', 'untracked', 'Android', 

In [197]:
test.drop(labels=['date_account_created'], axis=1, inplace=True)
test.drop(labels=['timestamp_first_active'], axis=1, inplace=True)
test.drop(labels=['date_first_booking'], axis=1, inplace=True)
test.drop(labels=['gender'], axis=1, inplace=True)
test.drop(labels=['signup_method'], axis=1, inplace=True)
test.drop(labels=['language'], axis=1, inplace=True)
test.drop(labels=['affiliate_channel'], axis=1, inplace=True)
test.drop(labels=['affiliate_provider'], axis=1, inplace=True)
test.drop(labels=['first_affiliate_tracked'], axis=1, inplace=True)
test.drop(labels=['signup_app'], axis=1, inplace=True)
test.drop(labels=['first_device_type'], axis=1, inplace=True)
test.drop(labels=['first_browser'], axis=1, inplace=True)

In [198]:
test['age'] = test['age'].apply(lambda x: int(x))

In [199]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62096 entries, 0 to 62095
Columns: 119 entries, idtest to wOSBrowser
dtypes: float64(117), int64(1), object(1)
memory usage: 56.9+ MB


In [200]:
test.head()

,idtest,age,signup_flow,dac_dayofweek,dac_day,dac_month,dac_year,tfa_dayofweek,tfa_day,tfa_month,...,Pale Moon,Safari,SeaMonkey,Silk,SiteKiosk,Sogou Explorer,UC Browser,Yandex.Browser,notknown,wOSBrowser
0,5uwns89zht,35,0,1,182,7,2014,1,182,7,...,0,0,0,0,0,0,0,0,0,0
1,jtl0dijy2j,34,0,1,182,7,2014,1,182,7,...,0,0,0,0,0,0,0,0,0,0
2,xx0ulgorjt,34,0,1,182,7,2014,1,182,7,...,0,0,0,0,0,0,0,0,0,0
3,6c6puo6ix0,34,0,1,182,7,2014,1,182,7,...,0,0,0,0,0,0,0,0,0,0
4,czqhjk3yfe,34,0,1,182,7,2014,1,182,7,...,0,1,0,0,0,0,0,0,0,0


In [201]:
feature_cols = ['age', 'signup_flow', 'dac_dayofweek', 'dac_day', 'dac_month', 'dac_year', 'tfa_dayofweek', \
                'tfa_day', 'tfa_month', 'tfa_year', 'tfa_hour', '-unknown-test', 'FEMALE', 'MALE', 'OTHER', 'basic', 'facebooktest', \
                'ca', 'cs', 'da', 'de', 'el', 'en', 'es', 'fi', 'fr', 'hu', 'id', 'it', 'ja', 'ko', 'nl', \
                'no', 'pl', 'pt', 'ru', 'sv', 'th', 'tr', 'zh', 'content', 'directtest', 'othertest', \
                'remarketing', 'sem-brand', 'sem-non-brand', 'seo', 'baidu', 'bing', 'craigslist', 'direct', \
                'email-marketing', 'facebook', 'facebook-open-graph', 'google', 'gsp', 'meetup', 'naver', 'other', \
                'padmapper', 'vast', 'yahoo', 'yandex', 'linked', 'local ops', 'marketing', 'omg', 'product', \
                'tracked-other', 'untracked', 'Android', 'Moweb', 'Web', 'iOS', 'Android Phone', 'Android Tablet', \
                'Desktop (Other)', 'Mac Desktop', 'Other/Unknown', 'SmartPhone (Other)', 'Windows Desktop', 'iPad', \
                'iPhone', 'AOL Explorer', 'Android Browser', 'Apple Mail', \
                'BlackBerry Browser', 'Chrome', 'Chrome Mobile', 'Chromium', 'CometBird', 'Firefox', \
                'IE', 'IE Mobile', 'IceWeasel', 'Iron', 'Maxthon', 'Mobile Firefox', \
                'Mobile Safari', 'Opera', 'Opera Mini', 'Opera Mobile', \
                'Pale Moon', 'Safari', 'SeaMonkey', 'Silk', 'SiteKiosk', 'Sogou Explorer', \
                'Yandex.Browser', 'notknown', 'wOSBrowser']
X = test[feature_cols]
X.shape

(62096, 111)

In [202]:
X = X.fillna(0)

In [203]:
X.isnull().any().any()

False

In [204]:
X.head()

,age,signup_flow,dac_dayofweek,dac_day,dac_month,dac_year,tfa_dayofweek,tfa_day,tfa_month,tfa_year,...,Opera Mobile,Pale Moon,Safari,SeaMonkey,Silk,SiteKiosk,Sogou Explorer,Yandex.Browser,notknown,wOSBrowser
0,35,0,1,182,7,2014,1,182,7,2014,...,0,0,0,0,0,0,0,0,0,0
1,34,0,1,182,7,2014,1,182,7,2014,...,0,0,0,0,0,0,0,0,0,0
2,34,0,1,182,7,2014,1,182,7,2014,...,0,0,0,0,0,0,0,0,0,0
3,34,0,1,182,7,2014,1,182,7,2014,...,0,0,0,0,0,0,0,0,0,0
4,34,0,1,182,7,2014,1,182,7,2014,...,0,0,1,0,0,0,0,0,0,0


In [205]:
from sklearn.externals import joblib
knn = joblib.load('airbnb-knn.pkl') 

In [206]:
firstknnpredict = knn.predict(X)
firstknnpredict

array([ 7, 10, 10, ..., 10,  7,  7])

In [207]:
rfr = joblib.load('airbnb-rfr.pkl')

In [208]:
firstrfrpredict = rfr.predict(X)
firstrfrpredict

array([ 7,  7, 10, ..., 10, 10,  7])

In [1]:
#Put it into correct format in order to post to Kaggle.

In [209]:
le = joblib.load('airbnb-le.pkl')

In [210]:
invknn = le.inverse_transform(firstknnpredict)
invknn

array(['NDF', 'US', 'US', ..., 'US', 'NDF', 'NDF'], 
      dtype='<U5')

In [211]:
wonder = list(invknn)

In [212]:
wonder

['NDF',
 'US',
 'US',
 'US',
 'US',
 'NDF',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'NDF',
 'US',
 'US',
 'NDF',
 'US',
 'NDF',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'NDF',
 'US',
 'US',
 'US',
 'NDF',
 'NDF',
 'US',
 'NDF',
 'NDF',
 'US',
 'NDF',
 'FR',
 'NDF',
 'US',
 'US',
 'US',
 'US',
 'other',
 'NDF',
 'US',
 'US',
 'NDF',
 'US',
 'US',
 'US',
 'NDF',
 'US',
 'NDF',
 'US',
 'other',
 'US',
 'NDF',
 'US',
 'US',
 'NDF',
 'other',
 'US',
 'US',
 'US',
 'US',
 'NDF',
 'other',
 'NDF',
 'NDF',
 'NDF',
 'nan',
 'other',
 'NDF',
 'NDF',
 'US',
 'NDF',
 'US',
 'NDF',
 'US',
 'NDF',
 'US',
 'ES',
 'NDF',
 'NDF',
 'US',
 'NDF',
 'NDF',
 'US',
 'US',
 'NDF',
 'NDF',
 'US',
 'NDF',
 'US',
 'other',
 'US',
 'US',
 'US',
 'US',
 'US',
 'US',
 'NDF',
 'NDF',
 'US',
 'US',
 'NDF',
 'NDF',
 'US',
 'US',
 'NDF',
 'NDF',
 'NDF',
 'other',
 'US',
 'NDF',
 'NDF',
 'US',
 'US',
 'NDF',
 'NDF',
 'US',
 'US',
 'NDF',
 'US',
 'NDF',
 'NDF',
 'NDF'

In [213]:
soleiltest = test['idtest']

In [214]:
soleiltest

0        5uwns89zht
1        jtl0dijy2j
2        xx0ulgorjt
3        6c6puo6ix0
4        czqhjk3yfe
5        szx28ujmhf
6        guenkfjcbq
7        tkpq0mlugk
8        3xtgd5p9dn
9        md9aj22l5a
10       gg3eswjxdf
11       fyomoivygn
12       iq4kkd5oan
13       6k1xls6x5j
14       jodmb2ok1f
15       eq6fy0m4vc
16       yq4i7nfh6l
17       q5pibqdous
18       i0sc6d3j8s
19       br5mcrsqzn
20       rddbczuxx1
21       glck7hlmzz
22       sxpkaxep8n
23       sr4ntmalz2
24       f6wueq1ccn
25       ovc6nwn6mj
26       n10skstp90
27       5jrbdigmv4
28       d45nngmojp
29       y0frb6t1kq
            ...    
62066    bsv2ev628t
62067    06echc56pl
62068    niqgaye2ov
62069    61iwzuhw6e
62070    gks02el96u
62071    v4r1161l0r
62072    p1clbqd0o6
62073    ozb2z0km6l
62074    w3e3sp6i70
62075    q5bxbq0asg
62076    1xa5t3t0la
62077    zuvz7gfpjz
62078    gpusl6ppgf
62079    gpijioh4eh
62080    3ptlvdxss9
62081    f9a1ncjnrg
62082    kofaz2kh70
62083    6xrmom7hjo
62084    cg9wqgnad2


In [215]:
#soleiltest = [test['idtest']]

In [216]:
addcol = invknn.tolist()

In [217]:
#soleiltest
#addcol
print(type(soleiltest))
print(type(addcol))
print(type(wonder))
print(len(addcol))
print(len(soleiltest))
print(len(wonder))

<class 'pandas.core.series.Series'>
<class 'list'>
<class 'list'>
62096
62096
62096


In [228]:
soleil_data = pd.DataFrame({'id': soleiltest, 'country': wonder})

In [230]:
print(soleil_data)

      country          id
0         NDF  5uwns89zht
1          US  jtl0dijy2j
2          US  xx0ulgorjt
3          US  6c6puo6ix0
4          US  czqhjk3yfe
5         NDF  szx28ujmhf
6          US  guenkfjcbq
7          US  tkpq0mlugk
8          US  3xtgd5p9dn
9          US  md9aj22l5a
10         US  gg3eswjxdf
11         US  fyomoivygn
12         US  iq4kkd5oan
13         US  6k1xls6x5j
14         US  jodmb2ok1f
15        NDF  eq6fy0m4vc
16         US  yq4i7nfh6l
17         US  q5pibqdous
18        NDF  i0sc6d3j8s
19         US  br5mcrsqzn
20        NDF  rddbczuxx1
21         US  glck7hlmzz
22         US  sxpkaxep8n
23         US  sr4ntmalz2
24         US  f6wueq1ccn
25         US  ovc6nwn6mj
26         US  n10skstp90
27         US  5jrbdigmv4
28         US  d45nngmojp
29         US  y0frb6t1kq
...       ...         ...
62066     NDF  bsv2ev628t
62067     NDF  06echc56pl
62068      US  niqgaye2ov
62069     NDF  61iwzuhw6e
62070     NDF  gks02el96u
62071      US  v4r1161l0r
62072     ND

In [231]:
cols = soleil_data.columns.tolist()

In [232]:
cols

['country', 'id']

In [233]:
cols = ['id', 'country']

In [234]:
soleil_data = soleil_data[cols]

In [235]:
soleil_data

,id,country
0,5uwns89zht,NDF
1,jtl0dijy2j,US
2,xx0ulgorjt,US
3,6c6puo6ix0,US
4,czqhjk3yfe,US
5,szx28ujmhf,NDF
6,guenkfjcbq,US
7,tkpq0mlugk,US
8,3xtgd5p9dn,US
9,md9aj22l5a,US


In [247]:
soleil_data.isnull().sum()

id         308
country      0
dtype: int64

In [255]:
pd.isnull(soleil_data)

,id,country
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
5,False,False
6,False,False
7,False,False
8,False,False
9,False,False


In [263]:
soleil_data = soleil_data.dropna()

In [264]:
soleil_data

,id,country
0,5uwns89zht,NDF
1,jtl0dijy2j,US
2,xx0ulgorjt,US
3,6c6puo6ix0,US
4,czqhjk3yfe,US
5,szx28ujmhf,NDF
6,guenkfjcbq,US
7,tkpq0mlugk,US
8,3xtgd5p9dn,US
9,md9aj22l5a,US


In [265]:
soleil_data.to_csv('soleil_data1.csv', index=False)

In [266]:
hi = pd.read_csv('soleil_data1.csv')

In [267]:
hi

,id,country
0,5uwns89zht,NDF
1,jtl0dijy2j,US
2,xx0ulgorjt,US
3,6c6puo6ix0,US
4,czqhjk3yfe,US
5,szx28ujmhf,NDF
6,guenkfjcbq,US
7,tkpq0mlugk,US
8,3xtgd5p9dn,US
9,md9aj22l5a,US
